# MNIST Example

This example loads 10 mnist images from a subdirectory and classifies them using a W1A2 ConvNet based on QNN-MO-PYNQ.


In [1]:
import os, pickle, random
from datetime import datetime
from matplotlib import pyplot as plt
from PIL import Image
#%matplotlib inline

import numpy as np
import cv2

import qnn
from qnn import Dorefanet
from qnn import utils

import sys
import scipy.io
import json

from cs231n.fast_layers import conv_forward_fast, max_pool_forward_fast 

def exiter():
    classifier.deinit_accelerator()
    from pynq import Xlnk
    xlnk = Xlnk();
    xlnk.xlnk_reset() 
    

Load the classifier.

In [2]:
classifier = Dorefanet()
net = classifier.load_network(json_layer="/home/xilinx/jupyter_notebooks/QNN/qnn/params/mnist.json")

json_network_custom = "/home/xilinx/jupyter_notebooks/QNN/qnn/bitstreams/bitmnist.json"
json_layer_custom =   "/home/xilinx/jupyter_notebooks/QNN/qnn/params/mnist.json"
bit_file_custom =     "/home/xilinx/jupyter_notebooks/QNN/qnn/bitstreams/mnist.bit"

classifier.init_accelerator(bit_file=bit_file_custom, json_network=json_network_custom, json_layer=json_layer_custom)

Load the weights for the first conv layer and last FC layers.

In [3]:
conv0_W = np.load('/home/xilinx/jupyter_notebooks/qnnparams/mnist2/binparam-mnist/conv0/W0.npy')
conv0_b = np.load('/home/xilinx/jupyter_notebooks/qnnparams/mnist2/binparam-mnist/conv0/b0.npy')
conv0_W = np.swapaxes(conv0_W,0,3)
conv0_W = np.swapaxes(conv0_W,1,2)
conv0_W = np.swapaxes(conv0_W,2,3)

activation_bits = 2
th = 2**activation_bits-1
conv0_T = range(th) 
conv0_T = np.asarray(conv0_T,dtype=np.float32) 
conv0_T += 0.5
conv0_T /= th;
 
fc0_W = np.load('/home/xilinx/jupyter_notebooks/QNN/qnn/params/binparam-mnist/fc0/W0.npy')
fc0_b = np.load('/home/xilinx/jupyter_notebooks/QNN/qnn/params/binparam-mnist/fc0/b0.npy')
fc1_W = np.load('/home/xilinx/jupyter_notebooks/QNN/qnn/params/binparam-mnist/fc1/W0.npy')
fc1_b = np.load('/home/xilinx/jupyter_notebooks/QNN/qnn/params/binparam-mnist/fc1/b0.npy')

out_dim = net['conv3']['output'][1]
out_ch =  net['conv3']['output'][0]

Now loop over and classify images.

In [4]:
idx = 0
correct = 0 
cur_time = 0;
total_time = 0;

directory = 'mnist_test/'

for root, dirs, files in os.walk(directory):
  for name in files: 
    curr_class = name.split('.')[-2].split('-')[-1][3]
    #print(filename)
    img_file = os.path.join(root, name)
    img = cv2.imread(img_file,0).astype('float32')
    img = img / 255.0
    
    startx = datetime.now()
    
    img = img [np.newaxis,:,:,np.newaxis]
    idx += 1
       
     
    img = np.rollaxis(img,3, 1)
    conv_param = {'stride': 1, 'pad': 1}
    conv0, _ = conv_forward_fast(img, conv0_W, conv0_b, conv_param)  
    
     
    pool_param = {'pool_height': 2, 'pool_width': 2, 'stride': 2}
    conv0, _ = max_pool_forward_fast(conv0, pool_param)    
    conv0 = conv0.reshape((64, 14, 14))     
       
    # The result in then quantized to 2 bits representation for the subsequent HW offload
    # conv0 = np.clip(conv0,0,1) # not necessary
    conv0 = utils.threshold(conv0, conv0_T)

    # Compute offloaded convolutional layers

    conv_output = classifier.get_accel_buffer(out_ch, out_dim);
    conv_input = classifier.prepare_buffer(conv0)
    
    classifier.inference(conv_input, conv_output)

    conv_output = classifier.postprocess_buffer(conv_output)
    fc0_out = utils.fully_connected(conv_output, fc0_W, fc0_b)
    fc1_out = utils.fully_connected(fc0_out, fc1_W, fc1_b)

    predict = np.argmax(fc1_out)
    
    endx = datetime.now()
    cur_time = (endx - startx).total_seconds()
    total_time += cur_time
        
    if predict == int(curr_class):
        correct += 1    
                        
    print('#{4}  {2}    Class {0}  , Predicted {1},    Accuracy {3:.2f}'.format(curr_class,str(predict),name,correct/idx*100,idx))

    if idx == 10:
        avg_time = total_time / idx * 1000000
        print('Correct: {0} / {1}'.format(correct,idx))
        print('Average Time: {0} ms'.format(avg_time/1000))
        print('Fps: {0} \n\n'.format(1/avg_time*1e6)) 
        
        exiter()
       
exiter()

#1  000015-num5.png    Class 5  , Predicted 5,    Accuracy 100.00
#2  000003-num0.png    Class 0  , Predicted 0,    Accuracy 100.00
#3  000063-num3.png    Class 3  , Predicted 3,    Accuracy 100.00
#4  000081-num6.png    Class 6  , Predicted 6,    Accuracy 100.00
#5  000002-num1.png    Class 1  , Predicted 1,    Accuracy 100.00
#6  000000-num7.png    Class 7  , Predicted 7,    Accuracy 100.00
#7  000061-num8.png    Class 8  , Predicted 8,    Accuracy 100.00
#8  000001-num2.png    Class 2  , Predicted 2,    Accuracy 100.00
#9  000007-num9.png    Class 9  , Predicted 9,    Accuracy 100.00
#10  000004-num4.png    Class 4  , Predicted 4,    Accuracy 100.00
Correct: 10 / 10
Average Time: 42.3693 ms
Fps: 23.601994840603925 


